In [46]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader as api
import pprint
import tqdm

pp = pprint.PrettyPrinter()

## train by gensim

In [47]:
dataset = api.load("text8")
model = Word2Vec(dataset)

In [32]:
print(model.wv['king'])

[ 2.0953395e+00 -1.9731241e+00  2.6552367e-01 -1.7134460e+00
 -3.5115216e-02 -1.5850774e+00  9.1741782e-01  8.8943012e-02
 -1.0691686e+00  2.9206662e+00  1.1215485e+00 -1.5152329e+00
  7.9389250e-01  2.8436303e-01  1.0348152e+00  9.0016192e-01
 -1.1116505e-01  2.1811137e+00  2.8085884e-01  5.5462706e-01
 -9.2958248e-01 -2.0932276e+00 -2.3815374e+00 -6.8129760e-01
  2.1442430e+00  4.0424764e-01  2.0032477e+00 -1.3737109e+00
  1.0573367e+00  1.1310998e+00  1.5471361e+00  1.3263985e+00
 -6.0701984e-01  4.0566288e-03 -1.9202086e+00  9.5603365e-01
 -2.4572344e+00  1.5418106e+00 -1.5725791e-01 -1.5049566e+00
  3.7481868e+00  2.8140945e+00  1.4546834e+00 -3.9601088e-01
 -1.8088385e+00 -1.1165047e+00 -1.9582524e+00  1.8460208e+00
  8.0703312e-01 -8.0348325e-01 -3.2950044e+00 -4.7357445e+00
  3.9479067e+00 -5.1670325e-01  1.2630069e-01  6.3440585e-01
 -1.5541737e+00 -1.6077358e+00  1.4785161e+00  5.5884087e-01
 -1.0952820e+00 -1.4584171e+00  2.4845860e+00  1.5445882e+00
  2.0742581e+00 -3.04112

In [33]:
print(model.wv.evaluate_word_pairs(datapath('wordsim353.tsv')))
print(model.wv.evaluate_word_pairs(datapath('simlex999.txt')))

((0.6171452294174373, 3.191455754669189e-38), SpearmanrResult(correlation=0.6303181401158278, pvalue=2.8598067150066397e-40), 0.56657223796034)
((0.26859698827550466, 7.451789773933623e-18), SpearmanrResult(correlation=0.24889625331203918, pvalue=1.793141466969908e-15), 0.7007007007007007)


## train by word2vec diy

In [44]:
dataset = api.load("text8")
with open("dataset/text8.txt", "w") as f:
    for data in dataset:
        f.write(" ".join(data) + "\n")

In [45]:
from word2vec.trainer import Word2VecTrainer

%load_ext autoreload
%autoreload 2
# corpus_path = datapath('lee_background.cor')
corpus_path = "dataset/text8.txt"
w2v = Word2VecTrainer(input_file=corpus_path, output_file="out.vec", epochs=3)
print("Training...")
w2v.train()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Read 1 M words.
Read 2 M words.
Read 3 M words.
Read 4 M words.
Read 5 M words.
Read 6 M words.
Read 7 M words.
Read 8 M words.
Read 9 M words.
Read 10 M words.
Read 11 M words.
Read 12 M words.
Read 13 M words.
Read 14 M words.
Read 15 M words.
Read 16 M words.
Read 17 M words.
Total embeddings: 42432


TypeError: super(type, obj): obj must be an instance or subtype of type

In [22]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("out.vec", binary=False)
vocab = list(wv.vocab.keys())
print("Loaded vocab size %i" % len(vocab))

Loaded vocab size 662


In [23]:
print(wv.evaluate_word_pairs(datapath('wordsim353.tsv')))
print(wv.evaluate_word_pairs(datapath('simlex999.txt')))

((-0.38647145100632446, 0.3442872432184244), SpearmanrResult(correlation=-0.2142857142857143, pvalue=0.6103444156452669), 97.73371104815864)
((-0.5657293486038087, 0.022365071783484673), SpearmanrResult(correlation=-0.4911764705882353, pvalue=0.053350435677943356), 98.3983983983984)
